# Preprocessing

The following notebook provides the preprocessing steps undertaken in the project.

The first step involves importing the necessary libraries; matplotlib.pyplot, numpy, pandas, LinearRegression from sklearn.linear_model and geopandas.

In [46]:
"""
Created on Sun Jul 23 06:45:14 2023

@author: Jordanius
"""

# Import the required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Read in the data
Next the road traffic accident dataset will be read in as a pandas DataFrame. The dimensions of the DataFrame are then printed (89911 road traffic accidents, 17 columns). We will also examine the first part of the dataset to see what information is contained in the columns.

In [47]:
# Read in Tasmanian Traffic Accident Dataset as pandas dataframe
crashes = pd.read_csv(r"C:\Users\jjrattle\Downloads\ArcGIS Pro Analysis 13092023\Preprocessing\Preprocessing\Crashes_2010-2023_July2023.csv")
print(crashes.shape)

# Examine a portion of the data
crashes.head()

(89811, 17)


,ID2,VCRN,UNIT_TYPE,DESCRIPTION,CRASH_DATE,CRASH_TIME,REPORT_DAT,SEVERITY,VISITED,SURFACE_TY,LIGHT_COND,CENTRE_LIN,SPEED_ZONE,LOCATION_D,Latitude,Longitude,Date2
0,30000953,NaN,LV,151 - Out of control,01-JAN-2010,10:00,1/1/10,Minor,Yes,Sealed,Daylight,Single broken,080,"Midland Highway, Dysart, Southern Midlands",-42.572891,147.214942,1/1/10
1,30011807,NaN,LV,171 - Left off carriageway into object or park...,01-JAN-2010,22:01,1/1/10,Minor,Yes,Sealed,Darkness (with street light),Single Continuous,060,"Hobart Road, Kings Meadows, Launceston",-41.461800,147.156263,1/1/10
2,30035158,NaN,LV; LV,144 - Parking vehicles only,01-JAN-2010,00:07,6/1/10,Property Damage Only,No,Sealed,Darkness (with street light),Single broken,050,"Off road at Bellerive, Clarence City",-42.874280,147.365210,1/1/10
3,30050366,NaN,LV; LV,149 - Other maneuvering,01-JAN-2010,08:45,1/1/10,Property Damage Only,No,Unsealed,Daylight,None,080,"Off road at Binalong Bay, Break O'Day",-41.255945,148.316701,1/1/10
4,30052829,NaN,LV,180 - Off carriageway right bend,01-JAN-2010,10:50,5/1/10,Minor,Yes,Sealed,Daylight,Double continuous,100,"Arthur Highway, Forcett, Sorell",-42.822408,147.679239,1/1/10


### Handling null values
The DataFrame will be checked to see if null values exist, and how many, in the columns of the DataFrame.

In [48]:
# Check for null values in the columns of the DataFrame
crashes.isnull().sum()

ID2                0
VCRN           20754
UNIT_TYPE         23
DESCRIPTION        0
CRASH_DATE         0
CRASH_TIME         0
REPORT_DAT         0
SEVERITY           0
VISITED            0
SURFACE_TY         0
LIGHT_COND         3
CENTRE_LIN         0
SPEED_ZONE         0
LOCATION_D       197
Latitude           0
Longitude          0
Date2              0
dtype: int64

VCRN, LOCATION_D, UNIT_TYPE and LIGHT_COND all contain null values.
VCRN, Vehicle Crash Registration Number, is not relevant in this study.
Also, REPORT_DAT does not provide additional information to the dataset as there is already CRASH_DATE and CRASH_TIME.
Both VCRN and REPORT_DAT will be removed from the DataFrame.

In [49]:
# Drop unused columns of Vehicle Customer Reference Number & the date the accident was reported
crashes = crashes.drop(columns = ['VCRN', 'REPORT_DAT'])

# check to see if there are any null values
print(crashes.isnull().sum())

ID2              0
UNIT_TYPE       23
DESCRIPTION      0
CRASH_DATE       0
CRASH_TIME       0
SEVERITY         0
VISITED          0
SURFACE_TY       0
LIGHT_COND       3
CENTRE_LIN       0
SPEED_ZONE       0
LOCATION_D     197
Latitude         0
Longitude        0
Date2            0
dtype: int64


There are still null values present in the DataFrame. 
Let's check a portion of the dataset where LOCATION_D has null values: 

In [50]:
# 
crashes[crashes['LOCATION_D'].isnull()].head()

,ID2,UNIT_TYPE,DESCRIPTION,CRASH_DATE,CRASH_TIME,SEVERITY,VISITED,SURFACE_TY,LIGHT_COND,CENTRE_LIN,SPEED_ZONE,LOCATION_D,Latitude,Longitude,Date2
24991,165789,LV; HV,130 - Vehicles in same lane/ rear end,05-DEC-2013,15:45,Property Damage Only,No,Sealed,Daylight,Double broken,Not known,NaN,0.0,0.0,5/12/13
25617,191908,LV; LV,131 - Vehicles in same lane/ left rear,10-JAN-2014,14:00,Property Damage Only,No,Sealed,Daylight,None,Not stated,NaN,0.0,0.0,10/1/14
25699,194829,LV; LV,149 - Other maneuvering,14-JAN-2014,18:00,Property Damage Only,No,Sealed,Daylight,None,<40,NaN,0.0,0.0,14/1/14
26050,209000,LV; LV,149 - Other maneuvering,03-FEB-2014,16:57,Property Damage Only,No,Not known,Not known,Not known,Not known,NaN,0.0,0.0,3/2/14
26368,220852,LV; LV,147 - Emerging from driveway or lane,20-FEB-2014,15:40,Property Damage Only,No,Sealed,Daylight,Single broken,050,NaN,0.0,0.0,20/2/14


As we can see, the Latitude and Longitude values are equal to 0 for these values. This most likely means that there was no, or an insufficient, location description provided for these road traffic accidents in the traffic accident report.

Let's see how many values of Latitude or Longitude = 0:

In [51]:
# Print the number of rows for each column where latitude or longitude equals zero
print(crashes.query('Latitude == 0 | Longitude == 0').count())

ID2            1149
UNIT_TYPE      1145
DESCRIPTION    1149
CRASH_DATE     1149
CRASH_TIME     1149
SEVERITY       1149
VISITED        1149
SURFACE_TY     1149
LIGHT_COND     1149
CENTRE_LIN     1149
SPEED_ZONE     1149
LOCATION_D      952
Latitude       1149
Longitude      1149
Date2          1149
dtype: int64


There seems to be 1149 records without correct coordinate information (except for some of the columns with null values).

It may be possible to determine the rough location of these traffic accidents from the location description 'LOCATION_D'.

Let's see how what values are given for LOCATION_D when Latitude = 0:

In [52]:
crashes[crashes['Latitude'] == 0]['LOCATION_D'].head()

23708    Not Available
24935    Not Available
24991              NaN
25074    Not Available
25181    Not Available
Name: LOCATION_D, dtype: object

It seems to be either 'Not Available' or null values. Let's count how many values:

In [53]:
crashes[crashes['Latitude'] == 0]['LOCATION_D'].value_counts()

Not available    945
Not Available      7
Name: LOCATION_D, dtype: int64

The values are either 'Not Available' (945) or 'Not available' (7), which equal to 952 as was seen above.

There does not seem to be any helpful information contained within this subset.

Let's check the traffic accident severity column 'SEVERITY':

In [54]:
crashes[crashes['Latitude'] == 0]['SEVERITY'].value_counts()

Property Damage Only    1143
Not known                  3
Minor                      2
Serious                    1
Name: SEVERITY, dtype: int64

The traffic accidents are mostly Property Damage Only incidents, with only two minor incidents and one serious incident.

Although some information will be lost, this information will not be vital to the study. Additionally, these records should be removed as they do not have any spatial information for the spatial analysis.

So we will remove the records where Latitude and Longitude = 0


In [55]:
# Drop traffic accidents where the latitude equals zero
crashes = crashes.drop(crashes[crashes['Latitude'] == 0].index)

Now that the DataFrame has been updated, let's check the null values again:

In [56]:
# check to see if there are any null values
print(crashes.isnull().sum())

ID2             0
UNIT_TYPE      19
DESCRIPTION     0
CRASH_DATE      0
CRASH_TIME      0
SEVERITY        0
VISITED         0
SURFACE_TY      0
LIGHT_COND      3
CENTRE_LIN      0
SPEED_ZONE      0
LOCATION_D      0
Latitude        0
Longitude       0
Date2           0
dtype: int64


Now let's check the first ten records where 'UNIT_TYPE' is null:

In [57]:
crashes[crashes['UNIT_TYPE'].isnull()].head(10)

,ID2,UNIT_TYPE,DESCRIPTION,CRASH_DATE,CRASH_TIME,SEVERITY,VISITED,SURFACE_TY,LIGHT_COND,CENTRE_LIN,SPEED_ZONE,LOCATION_D,Latitude,Longitude,Date2
80465,51639023,NaN,-,04-APR-2022,08:00,Property Damage Only,No,Sealed,Daylight,Single broken,100,"Bass Highway, Hellyer, Circular Head",-40.869924,145.433391,4/4/22
81284,51956421,NaN,160 - Parked,14-MAY-2022,22:25,Not known,Yes,Sealed,Darkness (with street light),Single broken,040,"Elizabeth Street, North Hobart, Hobart",-42.873392,147.315668,14/5/22
81285,51956428,NaN,160 - Parked,14-MAY-2022,22:25,Not known,Yes,Sealed,Darkness (with street light),Single broken,040,"Elizabeth Street, North Hobart, Hobart",-42.873392,147.315668,14/5/22
84572,51804671,NaN,-,29-OCT-2022,18:30,Property Damage Only,No,Sealed,Daylight,Not known,100,"Lyell Highway, Tarraleah, Central Highlands",-42.299034,146.458736,29/10/22
84937,51812507,NaN,-,15-NOV-2022,14:30,Property Damage Only,Yes,Sealed,Daylight,None,050,", New Town, Hobart",-42.864431,147.315802,15/11/22
85005,51814225,NaN,-,18-NOV-2022,16:15,Property Damage Only,Yes,Sealed,Daylight,None,100,"Exton Road, Exton, Meander Valley",-41.539372,146.731009,18/11/22
85592,51838877,NaN,-,14-DEC-2022,23:25,Property Damage Only,Yes,Sealed,Darkness (with street light),None,050,"null, Midway Point, Sorell",-42.796538,147.530818,14/12/22
86193,51874732,NaN,-,15-JAN-2023,11:30,Property Damage Only,No,Sealed,Daylight,Single broken,060,"Main Road, Binalong Bay, Break O'Day",-41.251463,148.307321,15/1/23
86498,51885394,NaN,-,31-JAN-2023,11:45,Property Damage Only,No,Sealed,Daylight,None,050,"Howick StreetMulgrave Street, West Launceston,...",-41.447208,147.144967,31/1/23
86845,51900321,NaN,-,16-FEB-2023,10:18,Property Damage Only,Yes,Sealed,Daylight,None,050,", Riverside, West Tamar",-41.421789,147.103371,16/2/23


These incidents are fairly recent and so may affect the spatiotemporal analysis if they are removed. So it maye be best to keep them.

The null values will be handled by using the pandas .fillna() function, which will be used to replace the null values with the string 'Unknown'.

In [59]:
crashes['UNIT_TYPE'] = crashes['UNIT_TYPE'].fillna(value = 'Unknown')
# check to see if there are any null values
crashes.isnull().sum()

ID2            0
UNIT_TYPE      0
DESCRIPTION    0
CRASH_DATE     0
CRASH_TIME     0
SEVERITY       0
VISITED        0
SURFACE_TY     0
LIGHT_COND     3
CENTRE_LIN     0
SPEED_ZONE     0
LOCATION_D     0
Latitude       0
Longitude      0
Date2          0
dtype: int64

We still have null values in the light conditions column 'LIGHT_COND'.
Let's examine the three records:

In [60]:
crashes[crashes['LIGHT_COND'].isnull()]

,ID2,UNIT_TYPE,DESCRIPTION,CRASH_DATE,CRASH_TIME,SEVERITY,VISITED,SURFACE_TY,LIGHT_COND,CENTRE_LIN,SPEED_ZONE,LOCATION_D,Latitude,Longitude,Date2
5993,30085311,LV,179 - Other straight,13-NOV-2010,01:09,Property Damage Only,Yes,Sealed,NaN,None,050,"Church Street, Launceston, Launceston",-41.448678,147.136220,13/11/10
9594,30124598,LV; LV,121 - Right through,21-MAY-2011,18:05,Property Damage Only,No,Sealed,NaN,Single broken,050,Intersection of King Street and Sandy Bay Road...,-42.894722,147.326638,21/5/11
16082,30148365,LV; LV,110 - Cross traffic,14-JUN-2012,19:18,Property Damage Only,Yes,Sealed,NaN,Single Continuous,050,Intersection of Frederick Street and Margaret ...,-41.443419,147.135169,14/6/12


This information could prove useful in the analysis and so will be kept. The null values will be filled in that same way as 'UNIT_TYPE':

In [61]:
crashes['LIGHT_COND'] = crashes['LIGHT_COND'].fillna(value = 'Unknown')
# check to see if there are any null values
crashes.isnull().sum()

ID2            0
UNIT_TYPE      0
DESCRIPTION    0
CRASH_DATE     0
CRASH_TIME     0
SEVERITY       0
VISITED        0
SURFACE_TY     0
LIGHT_COND     0
CENTRE_LIN     0
SPEED_ZONE     0
LOCATION_D     0
Latitude       0
Longitude      0
Date2          0
dtype: int64

Now there are no null values remaining in the DataFrame.

### Classifying Severity Values
Since this study will focus on traffic accident severity, it is important that the severity values are converted into the necessary format for the Hot Spot Analysis and Emerging Hot Spot Analysis tools, which both require numeric values as opposed to categorical.

The following code will create a new column called 'Severity Rank':

In [68]:
# Set up conditions for creating a new severity column
sev_con = [(crashes['SEVERITY'] == 'Fatal'), (crashes['SEVERITY'] == 'Serious'),
(crashes['SEVERITY'] == 'First Aid'), (crashes['SEVERITY'] == 'Minor'),
(crashes['SEVERITY'] == 'Property Damage Only'),(crashes['SEVERITY'] == 'Not known')]

# Define a list of severity values to match the conditions to
sev_cat = [5, 4, 3, 2, 1, 0]

# Creates a new column with numerical severity values
crashes['Severity Rank'] = np.select(sev_con, sev_cat)

Let's have a look at the sum of different severity values:

In [69]:
crashes['SEVERITY'].value_counts()

Property Damage Only    62037
Minor                   14196
First Aid                6122
Serious                  3133
Not known                2748
Fatal                     426
Name: SEVERITY, dtype: int64

'Not known' values are not useful for this study. 
With 2748 records having 'Not known' values, let's check what proportion of the data this is:

In [74]:
prop = crashes['SEVERITY'].value_counts()['Not known'] / crashes['SEVERITY'].count() * 100

print(f'Approximately {round(prop, 2)}% of the data will be removed if "Not known" values are removed.')

Approximately 3.1% of the data will be removed if "Not known" values are removed.


Information is important, however, relevant information is vital.
Although 3.1% of the data will be removed, this figure is not considered significant and it is also important to only include relevant data in the analysis.
Therefore we will now remove those records from the DataFrame and print its resulting dimensions:

In [75]:
# Drop the accidents where the Severity is not known
crashes.drop(crashes[crashes['SEVERITY'] == 'Not known'].index, inplace = True)
print(crashes.shape)

(85914, 16)


### Further Data Cleansing
During further inspections of the data it was noticed that there is one record that contains false date information.
This record will be removed below:

In [76]:
# Drop the last value in the dataset because it has a blunder in the date (year = 0043)
crashes = crashes.drop(crashes.index[-1])

### Create Datetime Column
To aid the creation of a time field in ArcGIS Pro, a new date column 'Acc_Date' will be added to the DataFrame that is a combination of 'CRASH_DATE' and 'CRASH_TIME'.
The new column is then converted into a datetime column:

In [77]:
crashes['Acc_Date'] = crashes['CRASH_DATE'] + ' ' + crashes['CRASH_TIME']
crashes['Acc_Date'] = pd.to_datetime(crashes['Acc_Date'], format = '%d-%b-%Y %H:%M')


### Export Data to be used in ArcGIS Pro
The initial preprocessing has now been conducted.
The DataFrame will now be exported as a CSV file to be used in the analysis.

In [78]:
crashes.to_csv(r'C:\Users\jjrattle\Downloads\ArcGIS Pro Analysis 13092023\Preprocessing\Preprocessing\Crash_Data_Cleaned.csv')

### Preprocessing stage now complete
The preprocessing has now been completed and the output CSV file can now be brought into ArcGIS Pro.